In [1]:
# get data lists
from nipype.interfaces.ants import ANTS
from nipype.interfaces import fsl
import os,time
OUT_DIR='/output/PD_ICA/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
atlas_9c='/templateflow/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c.nii'
atlas_9c_mask='/templateflow/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'
atlas_09_masked=OUT_DIR+'t1_MNI2009c_masked.nii.gz'
mask = fsl.ApplyMask(
    in_file=atlas_9c,
    out_file=atlas_09_masked,
    mask_file=atlas_9c_mask)
mask.run()

def dataGraber_sub(SUB_ID, DATA_DIR, TMPT_STR):
    import nipype.interfaces.io as nio
    import time
    t0=time.time()
    print('Grabbing files for: ', SUB_ID)
    OUT_FILE=[]
    out_len=len(TMPT_STR)
    if out_len == 0:
        print(SUB_ID+' has no files named: ', TMPT_STR)
        return OUT_FILE
    else:
        for i in range(out_len):
            TMP='%s/anat/%s_'+TMPT_STR[i]
            ds = nio.DataGrabber(infields=['subject_id', 'subject_id'])
            ds.inputs.base_directory = DATA_DIR # database
            ds.inputs.template = TMP 
            ds.inputs.subject_id = [SUB_ID]
            ds.inputs.sort_filelist = True
            res = ds.run()
            res_list = res.outputs.outfiles
            OUT_FILE.append(res_list)
        #print(SUB_ID+' files: ', OUT_FILE)
        print('dataGraber takes: ', time.time()-t0 )
        return OUT_FILE
def h5toWarp_nii(H5_FILE, OUT_DIR):
    from nipype.interfaces.ants import CompositeTransformUtil
    import os
    import time
    t0=time.time()
    tran = CompositeTransformUtil()
    tran.inputs.process = 'disassemble'
    tran.inputs.in_file = H5_FILE
    tran.inputs.out_file = OUT_DIR #bug
    #print(tran.cmdline)
    res=tran.run()
    out_warp  =OUT_DIR+'_Warp.nii.gz'
    out_affine=OUT_DIR+'_Affine.txt'
    os.system('mv '+res.outputs.displacement_field+' '+out_warp )
    os.system('mv '+res.outputs.affine_transform+' '+ out_affine)
    print('.h5 file disassemble takes: ', time.time()-t0 )
    return [out_warp, out_affine]
def GetJacobian_nii(IN_IMAGE,JACOB_IMG):
    # Jacobian of deformation field
    from nipype.interfaces.ants import CreateJacobianDeterminantImage
    import time
    t0=time.time()
    jacobian = CreateJacobianDeterminantImage()
    jacobian.inputs.imageDimension = 3
    jacobian.inputs.deformationField = IN_IMAGE
    jacobian.inputs.outputImage = JACOB_IMG
    jacobian.inputs.num_threads = 4
    #print(jacobian.cmdline)
    jacobian.run()
    print('Jacobian takes: ', time.time()-t0 )
    return 1

In [2]:
#PD_LIST='/codes/PD_template_subjects.list';
GROUPS=['PD','ET','NC']
SUB_LIST=[]; AGE_LIST=[];
for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    if not os.path.exists(current_OUT_DIR):
        os.makedirs(current_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [x[0:-1].split('\t')[1] for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(N_sub, sub_list, age_list)
    DATA_DIR='/data/'+current_group
    TMPT_list=['from-T1w_to-MNI152NLin2009cAsym_mode-image_xfm.h5', \
               'space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz', \
               'space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'];
    print('Processing ',current_group,' group start:')
    for i_sub in range(0, N_sub,1):
        import time
        t0=time.time()
        sub_str=sub_list[i_sub]
        print('Processing ',sub_str,' ( ',i_sub , ' ) start:')
        res1 = dataGraber_sub(sub_str, DATA_DIR, TMPT_list)
        H5_FILE=res1[0]
        WARP_OUT_DIR1=current_OUT_DIR+sub_str+'_'+'desc-preproc_T1w_space-MNI2009c'
        [out_warp, out_affine]=h5toWarp_nii(H5_FILE, WARP_OUT_DIR1)
        IN_IMAGE  = out_warp
        JACOB_IMG = current_OUT_DIR+sub_str+'_'+'desc-preproc_T1w_space-MNI2009c_Warp_Jacobian.nii.gz'
        GetJacobian_nii(out_warp, JACOB_IMG)
        print('Processing ',sub_str,' finished in ',time.time()-t0, '\n')

46 ['sub-0002', 'sub-0004', 'sub-0005', 'sub-0006', 'sub-0008', 'sub-0009', 'sub-0012', 'sub-0014', 'sub-0015', 'sub-0021', 'sub-0022', 'sub-0023', 'sub-0024', 'sub-0025', 'sub-0028', 'sub-0030', 'sub-0031', 'sub-0034', 'sub-0035', 'sub-0037', 'sub-0038', 'sub-0040', 'sub-0047', 'sub-0051', 'sub-0052', 'sub-0068', 'sub-0075', 'sub-0076', 'sub-0087', 'sub-0094', 'sub-0096', 'sub-0098', 'sub-0103', 'sub-0109', 'sub-0111', 'sub-0118', 'sub-0125', 'sub-0126', 'sub-0129', 'sub-0131', 'sub-0132', 'sub-0133', 'sub-0136', 'sub-1000', 'sub-1020', 'sub-4100'] ['70', '76', '45', '63', '59', '57', '47', '66', '65', '53', '62', '50', '63', '44', '61', '72', '51', '70', '69', '68', '54', '63', '75', '54', '56', '66', '66', '48', '57', '66', '73', '57', '62', '69', '69', '54', '60', '57', '69', '80', '74', '68', '62', '77', '66', '56']
Processing  PD  group start:
Processing  sub-0002  (  0  ) start:
Grabbing files for:  sub-0002
dataGraber takes:  0.006231069564819336
.h5 file disassemble takes:  29

In [9]:
with open('/codes/devel/PD_Marker/'+'PD'+'_info_ICA.list', 'r') as f_sub:
    sub_list= f_sub.readlines()
sub_list = [x[0:-1].split('\t')[0] for x in sub_list] # remove
print(sub_list)

['sub-0002', 'sub-0004', 'sub-0005', 'sub-0006', 'sub-0008', 'sub-0009', 'sub-0012', 'sub-0014', 'sub-0015', 'sub-0021', 'sub-0022', 'sub-0023', 'sub-0024', 'sub-0025', 'sub-0028', 'sub-0030', 'sub-0031', 'sub-0034', 'sub-0035', 'sub-0037', 'sub-0038', 'sub-0040', 'sub-0047', 'sub-0051', 'sub-0052', 'sub-0068', 'sub-0075', 'sub-0076', 'sub-0087', 'sub-0094', 'sub-0096', 'sub-0098', 'sub-0103', 'sub-0109', 'sub-0111', 'sub-0118', 'sub-0125', 'sub-0126', 'sub-0129', 'sub-0131', 'sub-0132', 'sub-0133', 'sub-0136', 'sub-1000', 'sub-1020', 'sub-4100']


In [3]:
# MELODIC ICA masked
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

t0=time.time()
g_path ='/output/sMRI/'
dir_all_jacobian = [g_path+x[-29:-24]+'_t1_masked_Jacobian_masked.nii.gz' for x in t1_list]
merged_file = g_path+'ICA_42_masked/42sub_30IC_masked_4d.nii.gz'
merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
#melodic_setup.inputs.no_bet = True
#melodic_setup.inputs.bg_threshold = 10
#melodic_setup.inputs.tr_sec = 1.5
#melodic_setup.inputs.mm_thresh = 0.5
#melodic_setup.inputs.out_stats = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.num_ICs = 30
#melodic_setup.inputs.t_des = 'timeDesign.mat'resICA
#melodic_setup.inputs.t_con = 'timeDesign.con'
#melodic_setup.inputs.s_des = 'subjectDesign.mat'
#melodic_setup.inputs.s_con = 'subjectDesign.con'
melodic_setup.inputs.out_dir = '/output/sMRI/ICA_42_masked'
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tMELODIC ICA takes: ', t2-t1)

Masked Jacobian:
 	Megring 4D file takes:  75.1306574344635
	MELODIC ICA takes:  1631.524489402771


In [64]:
# MELODIC ICA masked for PD vs HC
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

index_PD=[0,10,18,19,20,21,22,23,25,-1] #vt11 missing
index_ET=[1,2,3, 4, 5, 6,7,8,9,11,12,13,14,15,16,17,24,34,36,38,39,40] #fmr07,20,28,33,47,78,108,
# 112,145, 189; VT1-3,41,,42,43,109 missing 33 missing from the list
index_HC=[26,27,28,29,30,31,32,33,35] #vt22 missing

t0=time.time()
g_path ='/output/sMRI/'
dir_all_jacobian = [g_path+t1_list[x][-29:-24]+'_t1_masked_Jacobian_masked.nii.gz' for x in index_PD+index_HC]
merged_file = g_path+'ICA_PDHC_masked/PDHC_masked_4d.nii.gz'

merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('PD+HC Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
#melodic_setup.inputs.no_bet = True
#melodic_setup.inputs.bg_threshold = 10
#melodic_setup.inputs.tr_sec = 1.5
#melodic_setup.inputs.mm_thresh = 0.5
#melodic_setup.inputs.out_stats = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.num_ICs = 30
#melodic_setup.inputs.t_des = 'timeDesign.mat'resICA
#melodic_setup.inputs.t_con = 'timeDesign.con'
#melodic_setup.inputs.s_des = 'subjectDesign.mat'
#melodic_setup.inputs.s_con = 'subjectDesign.con'
melodic_setup.inputs.out_dir = '/output/sMRI/ICA_PDHC_masked'
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tPD+HC MELODIC ICA takes: ', t2-t1)

PD+HC Masked Jacobian:
 	Megring 4D file takes:  35.38829159736633
	PD+HC MELODIC ICA takes:  206.15899634361267


In [65]:
# MELODIC ICA masked for ET vs HC
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage

index_PD=[0,10,18,19,20,21,22,23,25,-1] #vt11 missing
index_ET=[1,2,3, 4, 5, 6,7,8,9,11,12,13,14,15,16,17,24,34,36,38,39,40] #fmr07,20,28,33,47,78,108,
# 112,145, 189; VT1-3,41,,42,43,109 missing 33 missing from the list
index_HC=[26,27,28,29,30,31,32,33,35] #vt22 missing

t0=time.time()
g_path ='/output/sMRI/'
dir_all_jacobian = [g_path+t1_list[x][-29:-24]+'_t1_masked_Jacobian_masked.nii.gz' for x in index_ET+index_HC]
merged_file = g_path+'ICA_ETHC_masked/ETHC_masked_4d.nii.gz'

merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('ET+HC Masked Jacobian:\n \tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
#melodic_setup.inputs.no_bet = True
#melodic_setup.inputs.bg_threshold = 10
#melodic_setup.inputs.tr_sec = 1.5
#melodic_setup.inputs.mm_thresh = 0.5
#melodic_setup.inputs.out_stats = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.num_ICs = 30
#melodic_setup.inputs.t_des = 'timeDesign.mat'resICA
#melodic_setup.inputs.t_con = 'timeDesign.con'
#melodic_setup.inputs.s_des = 'subjectDesign.mat'
#melodic_setup.inputs.s_con = 'subjectDesign.con'
melodic_setup.inputs.out_dir = '/output/sMRI/ICA_ETHC_masked'
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tET+HC MELODIC ICA takes: ', t2-t1)

ET+HC Masked Jacobian:
 	Megring 4D file takes:  54.842989683151245
	ET+HC MELODIC ICA takes:  1232.2800362110138


In [6]:
# MELODIC ICA no mask
t0=time.time()
g_path ='/output/sMRI/'
dir_all_jacobian = [g_path+x[-29:-24]+'_t1_masked_Jacobian.nii.gz' for x in t1_list]
merged_file = g_path+'ICA_42/42sub_30IC.nii.gz'
merger = fsl.Merge()
merger.inputs.in_files = dir_all_jacobian
merger.inputs.dimension = 'a'
merger.inputs.merged_file = merged_file
merger.cmdline
merger.run()
t1=time.time()
print('No mask Jacobian:\n\tMegring 4D file takes: ', t1-t0)
melodic_setup = fsl.MELODIC()
melodic_setup.inputs.approach = 'tica'
melodic_setup.inputs.in_files = [merged_file]
#melodic_setup.inputs.no_bet = True
#melodic_setup.inputs.bg_threshold = 10
#melodic_setup.inputs.tr_sec = 1.5
#melodic_setup.inputs.mm_thresh = 0.5
#melodic_setup.inputs.out_stats = True
melodic_setup.inputs.out_all = True
melodic_setup.inputs.num_ICs = 30
#melodic_setup.inputs.t_des = 'timeDesign.mat'resICA
#melodic_setup.inputs.t_con = 'timeDesign.con'
#melodic_setup.inputs.s_des = 'subjectDesign.mat'
#melodic_setup.inputs.s_con = 'subjectDesign.con'
melodic_setup.inputs.out_dir = '/output/sMRI/ICA_42'
melodic_setup.inputs.report = True
melodic_setup.cmdline
melodic_setup.run()
t2=time.time()
print('\tMELODIC ICA takes: ', t2-t1)

No mask Jacobian:
	Megring 4D file takes:  147.53803086280823
	MELODIC ICA takes:  16005.61508345604
